In [ ]:
import json
import pandas as pd

# Helper function to normalize date columns
def normalize_date_column(df, column):
    if column in df.columns:
        try:
            df[column] = pd.to_datetime(
                pd.json_normalize(df[column])['$date'],
                errors='coerce',
                unit='ms'
            ).astype('datetime64[s]')
        except Exception as e:
            print(f"Error normalizing {column}: {e}")

# Read JSON files
receipts_df = pd.read_json('receipts.json.gz', compression='gzip', lines=True)
users_df = pd.read_json('users.json.gz', compression='gzip', lines=True)
brands_df = pd.read_json('brands.json.gz', compression='gzip', lines=True)

# Users Dataset
users_df['lastLogin'] = users_df['lastLogin'].fillna(pd.NA)
users_df['_id'] = pd.json_normalize(users_df['_id'])
normalize_date_column(users_df, 'createdDate')
normalize_date_column(users_df, 'lastLogin')

# Brands Dataset
cpg_df = pd.json_normalize(brands_df['cpg']).add_prefix('cpg.')
norm_brands_df = brands_df.join(cpg_df, how='outer')
norm_brands_df['_id'] = pd.json_normalize(norm_brands_df['_id'])
norm_brands_df.drop(columns='cpg', axis=1, inplace=True)

# Receipts Dataset
receipts_df = receipts_df.explode('rewardsReceiptItemList').reset_index(drop=True)
receiptItemList_df = pd.json_normalize(receipts_df['rewardsReceiptItemList'], record_prefix='rewardsReceiptItemList_', errors='ignore')
norm_receipts_df = receipts_df.join(receiptItemList_df, how='outer')
norm_receipts_df.drop(columns=['rewardsReceiptItemList'], axis=1, inplace=True)

# Filling missing values in date columns
date_columns = ['createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for col in date_columns:
    norm_receipts_df[col] = norm_receipts_df[col].fillna(pd.NA)
    normalize_date_column(norm_receipts_df, col)

# Results Preview

print(users_df.head())

print(norm_brands_df.head())

print(norm_receipts_df.head())
